In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sqlalchemy as db

In [ ]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()
#shuffle data
# labeled_data_sampled = labeled_data.sample(n=10000, random_state=1)

In [ ]:

engine = db.create_engine('sqlite:///save_LC_procssed.db', echo=False)
sqlite_connection = engine.connect()
for i in range(1,10000):
    try:
        url = 'http://variables.cn:88/seldataz.php?SourceID=' + str(i)
    except:
        pass
    lc = pd.read_csv(url, header='infer')
    sqlite_table = labeled_data.ID[i]
    lc.to_sql(sqlite_table, sqlite_connection, if_exists='replace')
    print(i)
    